In [18]:
#Aruna IT20035808

#Intro to Random Forest Algorithm

Random forest is a supervised learning algorithm. It has two variations – one is used for classification problems and other is used for regression problems. It is one of the most flexible and easy to use algorithm. It creates decision trees on the given data samples, gets prediction from each tree and selects the best solution by means of voting. It is also a pretty good indicator of feature importance.

Random forest algorithm combines multiple decision-trees, resulting in a forest of trees, hence the name Random Forest. In the random forest classifier, the higher the number of trees in the forest results in higher accuracy.

In [19]:
#references
#https://www.kaggle.com/code/prashant111/random-forest-classifier-tutorial
#https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/FDM MINI PROJECT/Models

/content/drive/.shortcut-targets-by-id/1VOsnUh2YwiyJC0mAd6l0oM31b2bNe5GQ/FDM MINI PROJECT/Models


In [20]:
#imports

import os
import pandas as pd

# Ignore  the warnings
import warnings
warnings.filterwarnings('ignore')# data visualisation and manipulation

import matplotlib.pyplot as plt
#%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV

import numpy as np

In [21]:
print(os.getcwd())
print(os.listdir())

/content/drive/.shortcut-targets-by-id/1VOsnUh2YwiyJC0mAd6l0oM31b2bNe5GQ/FDM MINI PROJECT/Models
['Copy of Random_Forest_model.ipynb', 'final.csv', 'Random_Forest_model.ipynb', 'Final_hotel_bookings.csv']


In [22]:
#getting the data
df = pd.read_csv(os.getcwd() + '/Final_hotel_bookings.csv')
df.shape

(79330, 7)

In [27]:
#drop additional index column
df.drop(columns = df.columns[0], axis = 1, inplace= True)
df.head()

is_canceled  lead_time  stays_in_week_nights  agent  avg_daily_rate  \
0            0   0.009539                     2      6             0.0   
1            1   0.139905                     4      9            76.5   
2            1   0.103339                     4      9            68.0   
3            1   0.146264                     4      9            76.5   
4            1   0.158983                     2      9            76.5   

   total_of_special_requests  
0                          0  
1                          1  
2                          1  
3                          2  
4                          1

In [24]:
type(df)

pandas.core.frame.DataFrame

In [28]:
#get feature vector and target variable
x = df.drop('is_canceled', axis=1)
y = df['is_canceled']

In [29]:
#splitting dataset

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [30]:
#instantiate classifier without tuning

rfc = RandomForestClassifier(random_state = 1)
rfc.fit(x_train, y_train)


RandomForestClassifier(random_state=1)

In [31]:
y_train_pred = rfc.predict(x_train)
y_test_pred = rfc.predict(x_test)

In [32]:
print("training accuracy: ", accuracy_score(y_train, y_train_pred))
print("testing accuracy: ", accuracy_score(y_test, y_test_pred))

training accuracy:  0.9793088545136951
testing accuracy:  0.8319257111643347


untuned model fit shows signs of overfitting due to accuracy of training data > testing data

##Hyperparameter tuning using GridSearchCV cross validation

In [ ]:
#https://www.kaggle.com/code/sociopath00/random-forest-using-gridsearchcv
#https://www.kaggle.com/code/sociopath00/random-forest-using-gridsearchcv

In [33]:
#initialse the param_grid with a combination of hyper parameters

param_grid = { 
    'n_estimators': [300, 500, 800],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [35]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid = param_grid, cv = 3)

CV_rfc.fit(x_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=1),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [300, 500, 800]})

In [36]:
#get best set of hyper parameters
CV_rfc.best_params_

{'criterion': 'gini',
 'max_depth': 8,
 'max_features': 'auto',
 'n_estimators': 500}

In [45]:
#refit the model with best hyperparameters
rfc = RandomForestClassifier(random_state = 1, n_estimators = 500, 
                             max_features = 'auto', max_depth = 8, criterion = 'gini')
rfc.fit(x_train, y_train)

RandomForestClassifier(max_depth=8, n_estimators=500, random_state=1)

In [51]:
y_train_pred = rfc.predict(x_train)
y_test_pred = rfc.predict(x_test)

In [52]:
print("training accuracy: ", accuracy_score(y_train, y_train_pred))
print("testing accuracy: ", accuracy_score(y_test, y_test_pred))

training accuracy:  0.7790783526318633
testing accuracy:  0.7752426572545065


In [48]:
#https://towardsdatascience.com/random-forest-classification-678e551462f5

In [53]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

In [54]:
confusion_matrix(y_test, y_test_pred)

array([[11566,  2434],
       [ 2915,  6884]])

In [56]:
f1_score(y_test, y_test_pred)

0.7201966835800596

In [57]:
from sklearn.metrics import classification_report

In [59]:
#classification report

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.80      0.83      0.81     14000
           1       0.74      0.70      0.72      9799

    accuracy                           0.78     23799
   macro avg       0.77      0.76      0.77     23799
weighted avg       0.77      0.78      0.77     23799

